In [2]:
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_experimental.open_clip import OpenCLIPEmbeddings
import glob
import base64

paths = glob.glob('../images/*.jpeg', recursive=True)

In [3]:
lc_docs = []
def encode_image(path):
    with open(path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

for path in paths:
    doc = Document(
        page_content=encode_image(path),
        metadata ={
            'source': path
        }
    )

    lc_docs.append(doc)

In [4]:
vector_store = FAISS.from_documents(lc_docs, embedding=OpenCLIPEmbeddings())

In [5]:
retriever = vector_store.as_retriever()

In [11]:
dog_paths = glob.glob('../images/dog*.jpeg', recursive=True)

In [12]:
dog_to_cat = {}
for dog_pic in dog_paths:
    docs = retriever.invoke(encode_image(dog_pic))
    cats_retrieved = 0
    for i, doc in enumerate(docs):
        if "cat" in doc.metadata["source"]:
            cats_retrieved += 4 - i
dog_to_cat[dog_pic] = cats_retrieved

In [13]:
dog_to_cat

{'../images/dog_1.jpeg': 5,
 '../images/dog_3.jpeg': 3,
 '../images/dog_2.jpeg': 2,
 '../images/dog_5.jpeg': 1,
 '../images/dog_4.jpeg': 1}